# AdaNet

Cortes, Corinna, et al. "Adanet: Adaptive structural learning of artificial neural networks." arXiv preprint arXiv:1607.01097 (2016).

<img src="fig2.png" width=40%>

## Equation

### eq (5)

$$
F_t(w,u)=\frac{1}{m}\sum^m_{i=1} \Phi \left( 1-y_i(f_{t-1}(x_i)+w\cdot u(x_i) \right) + \Gamma_u ||w||_1
$$

- $\Phi$: Loss function
- $x_i$: data point
- $u \in {h, h'}$: 위 그림에서 두 가지 옵션 중 하나 (L, L+1).
- $w \in R^B$: 새로이 추가된 f weight (최종 아웃풋으로 연결되는 가중치)
- $f_{t-1}$: 이전 타임스텝 t-1 까지 생성한 네트워크
- $\Gamma_u = \lambda r_u + \beta$
    - $r_u$: u의 Rademacher complexity.

### eq (6)

$$
F_t(w,h)=\frac{1}{m}\sum^m_{i=1} \Phi \left( 1-y_i(f_{t-1}(x_i)+w\cdot h(x_i) \right) + R(w, h)
$$

- (5) 와 같지만 h 를 찾고 w 를 찾는 2-step 이 아니라 한번에 w 와 h 를 같이 찾는 방식.
- NN 을 사용할 때에는 이렇게 end-to-end 로 하는게 나음.
- 이 경우 R(w, h) 에는 원래의 w 에 대한 regularization term 뿐만 아니라 h 에 대한 regularization 도 새로이 들어간다:
    - $||h_s||_p \le \Lambda_{k,s}$

## Algorithm

```py
def AdaNet():
    for t in range(T):
        h1, w1 = WeakLearner(f[t-1], depth=L)
        h2, w2 = WeakLearner(f[t-1], depth=L+1)
        h, w = ArgMin(Loss(h1, w1), Loss(h2, w2))

        if Loss(f[t-1] + (h, w)) < Loss(f[t-1]):
            f[t] = f[t-1] + (h, w)
        else:
            return f[t-1]

    return f[T]
```

- 원래는 WeakLearner 에서 h 만 찾음. 그리고 optimize w.
    - 이 때 여기서 증명한 learning guarantee 는 w 를 최적화 할 때 적절하게 regularization 을 걸어서 트레이닝 로스가 좋아지면 밸리데이션 로스도 항상 좋아지도록 해줌
    - 또한 w 를 최적화하는 과정은 convex.
- 하지만 NN 으로 오면서 end-to-end 가 더 나을 것 같다 => h 와 w 를 같이 찾음
    - 이러면서 non-convex 가 되지만 수렴은 하므로 괜찮.
    - 이 때 w 에 대한 regularization 뿐만 아니라 h 에 대한 regularization 도 같이 걸어주자.

## Implementation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
torch.__version__

'0.4.1.post2'

In [4]:
class SubLinearNet(nn.Module):
    """ Sub-network builder for DNN """
    def __init__(self, n_inputs, n_units, n_layers, n_cons):
        """
        Args:
            n_inputs: size of input data
            n_units : # of hidden units
            n_layers: # of hidden layers
            n_cons: # of connections to embeddings
        """
        self.layers = nn.ModuleList()

        for i in range(n_layers):
            if i == 0: # first layer
                n_in = n_inputs
            else:
                n_in = n_units * n_cons
            linear_block = nn.Sequential([
                nn.Linear(n_in, n_units),
                #nn.Dropout(p=dropout, inplace=True),
                nn.ReLU(inplace=True)
            ])
            self.layers.append(linear_block)
        
        self.layers.append(nn.Linear())
        
    def forward(self, x, embeds):
        outs = [self.layers[0](x)]
        # embeds[0].cat(self.layers[0](x))
        for i, layer in enumerate(self.layers[1:], 1):
            inputs = embeds[i-1]
            out = layer(inputs)
            # embdes[i].cat(out)

#         return outs

In [11]:
t = [4,5,7]

In [8]:
t[-2+1:]

[3]

In [15]:
for i, v in enumerate(t, 2):
    print(i, v)

2 4
3 5
4 7
